In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

In [4]:
df1 = pd.read_csv("dictionary_verified_top_nsfw_words.csv")
df2 = pd.read_csv("dot05_nsfw_words.csv")

In [46]:
data = pd.read_csv("diffusion_db_unaltered.csv")
data = data.dropna()

In [127]:
df.head()

,word,count
0,knave,2
1,spake,6
2,wordsmith,4
3,tween,2
4,gou,3


In [131]:
stopwords = [
    "the", "a", "an", "is", "are", "and", "to", "in", "that", "have", "has",
    "with", "this", "these", "those", "it", "for", "on", "be", "was", "were",
    "can", "will", "would", "should", "may", "might", "do", "does", "did", "of"
]

# Assuming 'data' is a list of dictionaries with the key 'prompt'
filtered_prompts = []

for entry in data:
    prompt = entry.get('prompt', '')  # Get the prompt from each entry
    # Split the prompt into words, and filter out the stopwords
    filtered_prompt = [word for word in prompt.split() if word.lower() not in stopwords]
    entry['filtered_prompt'] = ' '.join(filtered_prompt)  # Add the filtered prompt to the entry

# Display the updated data with the filtered prompts
print(data[:5])  # Display the first 5 entries for example


[{'prompt': 'This is a scary movie with blood.', 'vector': 1, 'filtered_prompt': 'scary movie blood.'}, {'prompt': 'A robot walks in the city.', 'vector': 1, 'filtered_prompt': 'robot walks city.'}, {'prompt': 'She has beautiful eyes.', 'vector': 0, 'filtered_prompt': 'She beautiful eyes.'}, {'prompt': 'He is a bloodthirsty villain.', 'vector': 1, 'filtered_prompt': 'He bloodthirsty villain.'}]


In [132]:
# Example data structure for 'data'
data = [
    {"prompt": "This is a scary movie with blood."},
    {"prompt": "A robot walks in the city."},
    {"prompt": "She has beautiful eyes."},
    {"prompt": "He is a bloodthirsty villain."}
]

vectors = {
    'vector': []
}

stopwords = [
    "knave", "spake", "wordsmith", "tween", "gou", "gruelling", "underlight",
    "hogans", "racy", "skimpy", "sawhorse", "tucking", "tickled", "nevertheless",
    "whereas", "crunches", "reversi", "contortionist", "borning", "jockstrap",
    "supposedly", "bombshell", "virtue", "coiling", "gyal", "chaka", "midshipman",
    "cheongsam", "lamia", "orgy", "confess", "workaholic", "noncommittal", "dashiki",
    "pantyhose", "unclothed", "naiad", "haut", "cleavage", "unequivocal", "flustered",
    "placate", "cancerous", "cellist", "rocketeer", "damsel", "precariously", "deg",
    "merman", "impractical", "supine", "defiling", "fuddled", "squishing", "masseuse",
    "accusation", "ticklish", "sarong", "manliness", "gotch", "fakir", "clubbing",
    "infuriating", "histology", "knockers", "sugi", "forfeit", "expulsion", "fucking",
    "hoho", "thrashed", "difficulty", "nymphet", "lascivious", "pennyworth", "decolletage",
    "tejano", "swam", "retrieve", "reclined", "seducer", "kundalini", "interdependence",
    "dapped", "cadaver", "venere", "laugher", "naturist", "inhaler", "banya", "bearish",
    "invigorate", "intersex", "zaftig", "recumbent", "flagpole", "hydrosphere", "blading",
    "comely", "pulchritudinous", "undergarment", "dichotomy", "muddle", "lawless", "extant",
    "gallium", "stupendous", "seam", "lured", "similitude", "muumuu", "slain", "moonbow",
    "transgressive", "potbellied", "bachelorette", "temptress", "bacchanalian", "outtake",
    "concubine", "duchy", "busty", "skein", "conceive", "mestiza", "massaging", "nucha",
    "exotica", "waistband", "homoeroticism", "catatonic", "recession", "ramping", "weightlifter",
    "psychologic", "mangal", "motherly", "ultraviolent", "caulking", "notification", "pranking",
    "succubi", "cire", "cannister", "froglet", "batoon", "ripeness", "violoncello", "evicted",
    "divorced", "hermaphrodite", "await", "neigh", "asbestos", "charmed", "ravishing", "prower",
    "superfine", "recreating", "luxe", "gladiolus", "blackcoat", "archon", "demeanor", "sanatarium",
    "saxifrage", "sweeting", "titillation", "saron", "plastering", "xiii", "overtime", "pauper",
    "peekaboo", "perchance", "jiggle", "soared", "sitar", "sparring", "seductress", "rollers",
    "bratty", "cuffed", "hipshot", "combing", "endurance", "totality", "cervix", "gaining", "shrewd",
    "negotiator", "pilling", "gumshoe", "zaftig", "plucking", "frieze", "quantized", "wench", "allergic",
    "quench", "illusionist", "genteel", "recollection", "absorption", "evaluate", "incorrect", "ringer",
    "wink", "slain", "outcome", "pudendum", "hypnotising", "darkskin", "bustier", "shawls", "raunchy",
    "unclad", "peignoir", "ferme", "fixe", "unarmed", "sampling", "foresight", "intuit", "morth",
    "indifference", "musicality", "shyness", "amalgam", "eh", "tying", "blackguard", "planner", "mer",
    "nonchalant", "fellate", "unmentionables", "amorous", "retribution", "dominant", "purring", "fingered",
    "inordinate", "slathering", "leggy", "bifocals", "unpainted", "juxtapose", "supernumerary", "revising",
    "trademark", "mystified", "areola", "bittersweet", "inept", "erase", "flimsy", "raze", "snuggle",
    "skyrocket", "suntan", "unisex", "negligee", "plumper", "hedonic", "melodramatic", "barefooted",
    "woosh", "somniferous", "marksman", "rubby", "gull", "intimidated", "riviere", "unbound", "ichor",
    "baptismal", "multiracial", "gaed", "eugenics", "competent", "accentuate", "moly", "hurling", "cabling",
    "padi", "mediator", "opacity", "lifter", "rosed", "hurter", "clapperboard", "spoony", "nudist", "extatic",
    "pogrom"
]

keywords = {"boobs", "blood", "scary", "robot"}

# Iterate over the data and check for keywords or stopwords
for item in data:
    prompt = item['prompt']
    found_keyword = any(word in prompt for word in keywords)
    found_stopword = any(word in prompt for word in stopwords)
    
    if found_keyword or found_stopword:
        vectors['vector'].append(1)
    else:
        vectors['vector'].append(0)

# Adding the vector to the data
for i, item in enumerate(data):
    item['vector'] = vectors['vector'][i]

# Output the data with vectors
print(data)


[{'prompt': 'This is a scary movie with blood.', 'vector': 1}, {'prompt': 'A robot walks in the city.', 'vector': 1}, {'prompt': 'She has beautiful eyes.', 'vector': 0}, {'prompt': 'He is a bloodthirsty villain.', 'vector': 1}]


In [133]:
df.head()

,word,count
0,knave,2
1,spake,6
2,wordsmith,4
3,tween,2
4,gou,3


In [121]:
vocab = []
# Loop through the list of dictionaries (data is a list)
for i in range(len(data)):
    see = data[i]['prompt'].split()  # Accessing 'prompt' from each dictionary
    for word in see:
        if word not in vocab:
            vocab.append(word)

X = []
# Loop through again to create X
for i in range(len(data)):
    tsee = data[i]['prompt'].split()  # Accessing 'prompt' from each dictionary
    temp = []
    for word in tsee:
        if word in vocab:
            temp.append(vocab.index(word) + 1)  # Adding 1 to avoid zero indexing
    X.append(temp)

# Printing the result
print(X)


[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13], [14, 15, 16, 17], [18, 2, 3, 19, 20]]


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91564 entries, 0 to 91188
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    91564 non-null  object
 1   count   91564 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.1+ MB


In [5]:
df = pd.concat([df1, df2]).drop(columns=["Unnamed: 0"]).dropna()

In [65]:
df.head()

,word,count
0,knave,2
1,spake,6
2,wordsmith,4
3,tween,2
4,gou,3


In [39]:
vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(df['word'])

In [41]:
X_text

<91564x75400 sparse matrix of type '<class 'numpy.float64'>'
	with 91542 stored elements in Compressed Sparse Row format>

In [49]:
scaler = StandardScaler()
X_count = scaler.fit_transform(df[['count']])

In [50]:
X_features = [(X_text[i], X_count[i]) for i in range(len(df))]

In [51]:
y = np.array([0] * (len(df1)) + [1] * (len(df2)))


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [134]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91564 entries, 0 to 91188
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    91564 non-null  object
 1   count   91564 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.1+ MB


,word,count
0,knave,2
1,spake,6
2,wordsmith,4
3,tween,2
4,gou,3


In [94]:
import numpy as np
accuracy_score(np.array([[0, 1], [1, 1]]), np.ones((2, 2)))

0.5

In [101]:
from sklearn.metrics import f1_score

y_true = [[0, 0, 0], [1, 1, 1], [0, 1, 1]]
y_pred = [[0, 0, 0], [1, 1, 1], [1, 1, 0]]

# Compute F1-score for each label
f1 = f1_score(y_true, y_pred, average=None)

print(f1) 

[0.66666667 1.         0.66666667]


In [104]:
from sklearn.metrics import precision_score

# Example data (ensure `y` and `y_pred` are defined correctly)
y_true = [[0, 0, 0], [1, 1, 1], [0, 1, 1]]
y_pred = [[0, 0, 0], [1, 1, 1], [1, 1, 0]]

# Compute precision score
precision = precision_score(y_true, y_pred, average='macro')

print(precision)


0.8333333333333334


In [107]:
import numpy as np
from sklearn.metrics import recall_score

# Example data
y_true = np.array([[0, 0, 0], [1, 1, 1], [0, 1, 1]])
y_pred = np.array([[0, 0, 0], [1, 1, 1], [1, 1, 0]])

# Check shape before computing recall
print(y_true.shape, y_pred.shape)

# Compute recall score
recall = recall_score(y_true, y_pred, average='macro')
print("Recall Score:", recall)


(3, 3) (3, 3)
Recall Score: 0.8333333333333334


In [108]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to compute K-score
def knn_k_score(X_train, X_test, y_train, y_test, k):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Test function
k = 5  # Example value of K
score = knn_k_score(X_train, X_test, y_train, y_test, k)
print(f"Accuracy for K={k}: {score:.4f}")


Accuracy for K=5: 1.0000


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [ ]:
# List of harmful keywords (this can be expanded as needed)
harmful_keywords = ["violence", "abuse", "hate", "racist", "bullying", "threat"]

def is_harmful(prompt):
    # Check if any harmful keyword exists in the prompt
    for keyword in harmful_keywords:
        if keyword.lower() in prompt.lower():
            return True
    return False

# Input the prompt yourself
user_prompt = input("Enter a prompt: ")

# Check if the prompt is harmful or harmless
if is_harmful(user_prompt):
    print(f"Prompt: '{user_prompt}' is harmful.")
else:
    print(f"Prompt: '{user_prompt}' is harmless.")
